# IBM fairness framework 

This workbook demonstrates how to use IBM's AI Fairness 360 framework. The link to the git repo: https://github.com/Trusted-AI/AIF360 

The data has been generated superficially and has no empirical source. 

In [ ]:
!pip install -q aif360[all]

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

# Expanded Mock-up data with bias towards females
data = {
    'Age': [25, 45, 35, 50, 23, 30, 40, 28, 38, 48, 27, 37, 47, 26, 36, 46],
    'Income': [50000, 100000, 75000, 120000, 45000, 55000, 95000, 65000, 85000, 110000, 48000, 58000, 98000, 68000, 88000, 105000],
    'Gender': [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1],  # 1: Male, 0: Female
    'Hired': [1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1]   # 1: Hired, 0: Not Hired
}
df = pd.DataFrame(data)

In [9]:

# Split data into training and testing sets
train, test = train_test_split(df, test_size=0.2, random_state=42)

# Convert dataframes into BinaryLabelDataset format
train_bld = BinaryLabelDataset(df=train, label_names=['Hired'], protected_attribute_names=['Gender'])
test_bld = BinaryLabelDataset(df=test, label_names=['Hired'], protected_attribute_names=['Gender'])

# Compute fairness metric on original training dataset
metric_train_bld = BinaryLabelDatasetMetric(train_bld, unprivileged_groups=[{'Gender': 1}], privileged_groups=[{'Gender': 0}])
print(f'Original training dataset disparity: {metric_train_bld.mean_difference():.2f}')

# Mitigate bias by reweighing the dataset
RW = Reweighing(unprivileged_groups=[{'Gender': 1}], privileged_groups=[{'Gender': 0}])
train_bld_transformed = RW.fit_transform(train_bld)

# Compute fairness metric on transformed training dataset
metric_train_bld_transformed = BinaryLabelDatasetMetric(train_bld_transformed, unprivileged_groups=[{'Gender': 1}], privileged_groups=[{'Gender': 0}])
print(f'Transformed training dataset disparity: {metric_train_bld_transformed.mean_difference():.2f}')

Original training dataset disparity: 0.86
Transformed training dataset disparity: 0.50


In [10]:
# Train a classifier (e.g., logistic regression) on the transformed dataset
scaler = StandardScaler()
X_train = scaler.fit_transform(train_bld_transformed.features)
y_train = train_bld_transformed.labels.ravel()
clf = LogisticRegression().fit(X_train, y_train)

In [11]:
# 1. Transform the test data
X_test = scaler.transform(test_bld.features)
y_test = test_bld.labels.ravel()

# 2. Use the classifier to make predictions
y_pred = clf.predict(X_test)

# 3. Evaluate the classifier's performance
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Detailed classification report
report = classification_report(y_test, y_pred, target_names=["Not Hired", "Hired"])
print(report)

# If you want to evaluate fairness metrics on the test set:
test_bld_pred = test_bld.copy()
test_bld_pred.labels = y_pred.reshape(-1,1)
metric_test_bld = BinaryLabelDatasetMetric(test_bld_pred, unprivileged_groups=[{'Gender': 1}], privileged_groups=[{'Gender': 0}])
print("Test dataset disparity:", metric_test_bld.mean_difference())


Accuracy: 0.7500
              precision    recall  f1-score   support

   Not Hired       0.50      1.00      0.67         1
       Hired       1.00      0.67      0.80         3

    accuracy                           0.75         4
   macro avg       0.75      0.83      0.73         4
weighted avg       0.88      0.75      0.77         4

Test dataset disparity: 1.0
